In [23]:
from pynq import Overlay
import time
ol = Overlay('matrix.bit')

In [3]:
#ol?

In [24]:
dma = ol.axi_dma_0

In [25]:
data_send = dma.sendchannel
data_recv = dma.recvchannel

In [26]:
import numpy as np
import random 

In [28]:
random.seed(2)
data_A = np.random.uniform(low=1, high=2, size=(100, 100)).astype(float)
data_B = np.random.uniform(low=1, high=2, size=(100, 100)).astype(float)

In [29]:
data_A, data_B

(array([[1.02758367, 1.79277347, 1.96069385, ..., 1.62360982, 1.56664345,
         1.93705986],
        [1.46181429, 1.3144024 , 1.65516053, ..., 1.74166442, 1.28461952,
         1.46022058],
        [1.93624996, 1.23573704, 1.4886385 , ..., 1.47915547, 1.492076  ,
         1.10954058],
        ...,
        [1.35782461, 1.1347987 , 1.71530205, ..., 1.4761217 , 1.77429555,
         1.29492127],
        [1.89475049, 1.04328299, 1.15650205, ..., 1.91494901, 1.41000948,
         1.18962187],
        [1.5184021 , 1.59365653, 1.27262973, ..., 1.15259854, 1.91637053,
         1.61390598]]),
 array([[1.95660197, 1.27103886, 1.49315603, ..., 1.81484665, 1.51278007,
         1.7330087 ],
        [1.79521075, 1.1327877 , 1.58825854, ..., 1.93624692, 1.64872299,
         1.66518519],
        [1.34434921, 1.67453998, 1.41544589, ..., 1.84108073, 1.0862692 ,
         1.75905224],
        ...,
        [1.09095707, 1.81674104, 1.07816395, ..., 1.13886635, 1.30106798,
         1.92094713],
        [1.5

In [33]:
def multiply(matrix_a, matrix_b):
    result_matrix = np.zeros((100,100))
    for i in range(100):
        for j in range(100):
            for k in range(100):
                result_matrix[i][j] += matrix_a[i][k] * matrix_b[k][j]
                
    return result_matrix

In [34]:
start_time = time.time()
multiply(data_A, data_B) #software
end_time = time.time()
print(end_time - start_time)

6.23567795753479


In [35]:
from pynq import allocate

In [36]:
data_a_f = data_A.flatten()
data_b_f = data_B.flatten()

data_input = np.concatenate([data_a_f, data_b_f])

In [37]:
data_input

array([1.02758367, 1.79277347, 1.96069385, ..., 1.82437659, 1.31844467,
       1.36295549])

In [39]:
input_buffer = allocate(100*100*2, np.float32)
output_buffer = allocate((100,100), np.float32)

In [40]:
np.copyto(input_buffer, data_input)

In [43]:
start_time = time.time()
data_send.transfer(input_buffer)
data_recv.transfer(output_buffer)
data_send.wait()
data_recv.wait()
end_time = time.time()
print(end_time - start_time)

0.03267526626586914


In [44]:
output_buffer

PynqBuffer([[229.31499, 213.26712, 217.56096, ..., 221.48009, 224.40088,
             221.64177],
            [232.05087, 216.6436 , 221.45345, ..., 223.39395, 226.59868,
             224.28944],
            [237.70296, 221.67886, 228.51883, ..., 228.56053, 231.6273 ,
             231.04417],
            ...,
            [237.2951 , 223.83134, 227.9006 , ..., 229.28314, 232.39648,
             231.5579 ],
            [234.3505 , 217.34581, 223.46545, ..., 224.2848 , 226.56297,
             226.15886],
            [227.6924 , 210.71263, 217.46356, ..., 220.19905, 223.62665,
             220.36137]], dtype=float32)

In [45]:
diff = np.dot(data_A, data_B) - output_buffer

In [85]:
rms_error = np.sqrt(np.mean(diff**2)) 

In [60]:
diff, rms_error

(PynqBuffer([[-5.27744149e-06, -8.55481261e-07, -5.43558662e-06, ...,
               2.27535870e-05, -5.37467713e-07, -3.22216125e-05],
             [ 3.06554925e-05,  2.07101608e-05,  4.19613430e-05, ...,
               1.57229907e-05,  3.81117650e-06,  5.39301010e-06],
             [-4.00900539e-05,  4.63843638e-05, -1.41070602e-05, ...,
               2.79219779e-05, -5.04511490e-06, -4.97607423e-05],
             ...,
             [ 8.80234245e-06, -3.35164827e-07, -2.01015380e-05, ...,
              -4.83960164e-06,  1.97904280e-05, -1.07244557e-05],
             [ 6.14639336e-06,  1.48122664e-05,  1.37776703e-05, ...,
               1.08913228e-05,  2.43059203e-05, -1.00066157e-06],
             [ 3.15006646e-05, -3.46384275e-05, -1.46249834e-05, ...,
              -2.81820189e-05,  5.55518364e-05, -2.43192637e-05]]),
 PynqBuffer(3.1609277e-05))

In [61]:
%%time
np.dot(data_A, data_B)

CPU times: user 60.3 ms, sys: 0 ns, total: 60.3 ms
Wall time: 20.1 ms


array([[229.31498191, 213.26711951, 217.56095343, ..., 221.48011003,
        224.40087837, 221.64173719],
       [232.05090346, 216.64362117, 221.4534874 , ..., 223.39396714,
        226.5986824 , 224.28944936],
       [237.70291706, 221.67890991, 228.51881524, ..., 228.56055954,
        231.62729903, 231.04412443],
       ...,
       [237.29511378, 223.83134427, 227.90058415, ..., 229.28313725,
        232.39650417, 231.55789638],
       [234.35050053, 217.34582475, 223.46546788, ..., 224.28481619,
        226.56299733, 226.15885825],
       [227.69242957, 210.71259659, 217.46354739, ..., 220.19902272,
        223.6267035 , 220.36134958]])

In [86]:
rms_error

PynqBuffer(5.58996212e-06)